In [1]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore
#get_ipython().magic(u'matplotlib inline')
import cPickle
from multiprocessing import Pool
from sklearn import preprocessing



No handlers could be found for logger "gensim.models.word2vec"


In [7]:
X_train=pd.read_csv('X_train.csv')
Y_train=pd.read_csv('Y_train.csv')

df=X_train

(lenX,lenY)=np.shape(X_train.as_matrix())

non_num_columns=['AffinityCodeId','SelectedPackage','NameOfPolicyProduct','SCID'] # Complete
dropped_columns=['CustomerMD5Key','ReceivedDateTime','Unnamed: 0','TodayDate']


for column in non_num_columns:
    X_train[column]=le.fit_transform(X_train[column].as_matrix())

all_columns=df.columns.tolist()
for column_name in dropped_columns:
    all_columns.remove(column_name)

reduced_df=df[all_columns]

discrete_columns=['SelectedPackage','FirstDriverMaritalStatus','CarAnnualMileage']


In [ ]:
# Construct a dictionary
dictionary = corpora.Dictionary(equiv_texts)
dictionary.save('topics_alex.dict')
# print(dictionary)

In [26]:
corpus = [dictionary.doc2bow(text) for text in equiv_texts]
corpora.MmCorpus.serialize('topics_alex.mm', corpus)

In [22]:
#corpus=corpora.MmCorpus.load('topics_alex.mm')
mm = corpora.MmCorpus('topics_alex.mm')
id2word = corpora.Dictionary.load('topics_alex.dict')

In [24]:
train_LDA=True
n_topics=5
if train_LDA:
    lda = LdaMulticore(mm,id2word=id2word,num_topics=n_topics,workers=16)
    cPickle.dump(lda,open('lda_alex_'+str(n_topics)+'_topics.p','wb'))
else:
    lda=cPickle.load(open('lda_alex_'+str(n_topics)+'_topics.p','rb'))


In [11]:
def transform_vocab(idx_doc_bow):
    global lda,n_topics,mm
    bow_converted=lda[mm[idx_doc_bow]]
    local_array=[0]*(n_topics+1)
    for feat_tuple in bow_converted:
        local_array[feat_tuple[0]+1]=feat_tuple[1]
    return local_array

In [12]:

p=Pool(10)

L_indices=list(range(lenX))
        
# Compute the feature vector :
a=p.map(transform_vocab,L_indices)

In [20]:
cPickle.dump(a,open('sauvegarde_qui_sauve_la_vie.p','wb'))
topic_features_df=pd.DataFrame(a)
del a
topic_features_df=topic_features_df[topic_features_df.columns[1:]]
topic_features_df.columns=['lda_feature_'+str(i) for i in range(len(topic_features_df.columns))]

cPickle.dump(topic_features_df,open('lda_features_'+str(n_topics)+'topics_df.csv','wb'))

In [6]:
#global n_topics,ldamm

def dataframe2corpus(df,name_file):
    '''
    df : dataframe containing the data
    name_file : name of file saved containing the dictionnary for the lda model
    '''
    non_num_columns=['AffinityCodeId','SelectedPackage','NameOfPolicyProduct','SCID']
    le = preprocessing.LabelEncoder()
    for column in non_num_columns:
        df[column]=le.fit_transform(df[column].as_matrix())
    dropped_columns=['CustomerMD5Key','ReceivedDateTime','Unnamed: 0','TodayDate']
    all_columns=df.columns.tolist()
    for column_name in dropped_columns:
        all_columns.remove(column_name)

    reduced_df=df[all_columns]
    
    # List of column names with categorical variables
    categ_vars=[]
    for column in reduced_df.columns:
        if len(np.unique(reduced_df[column]))<50:    # we don't keep columns with too many different values
            categ_vars+=[column]

# Encode the data as a bag of words to feed gensim LDA
    equiv_texts=[]
    all_cat=reduced_df[categ_vars].as_matrix().tolist()
    for row in all_cat:
        text_row=[str(categ_vars[i])+':'+str(word) for (i,word) in enumerate(row)]
        equiv_texts+=[text_row]
    
    
    #dictionary = corpora.Dictionary(equiv_texts)
    #dictionary.save('topics_'+str(name_file)+'.dict') # dictionnary id2word
    dictionary=corpora.Dictionary.load('topics_alex.dict')
    
    
    
    corpus = [dictionary.doc2bow(text) for text in equiv_texts]
    corpora.MmCorpus.serialize('topics_'+str(name_file)+'.mm', corpus) # serialized dictionnary (for fast lda)
    return





In [7]:

import time

n_topics=5
t1=time.time()

name_file='train'


In [8]:
#X_test=pd.read_csv('X_test.csv')
X_train=pd.read_csv('X_train.csv')

df=X_train
del X_train

dataframe2corpus(df,name_file)
lenX=np.shape(df.as_matrix())[0]
del df


t2=time.time() # step 1 done : stored the vocabulary in the data
print(t2-t1)

/usr/local/lib/python2.7/dist-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


285.659765959


In [9]:
# step 2 : 
t2=time.time()


lda=cPickle.load(open('lda_alex_'+str(n_topics)+'_topics.p','rb')) # load lda model trained on the big corpus

mm_test = corpora.MmCorpus('topics_'+str(name_file)+'.mm')
id2word = corpora.Dictionary.load('topics_alex.dict')
   
#print('updating')
#lda.update(mm)
#print('ended updating')    
    


ldamm=lda[mm_test]

del mm_test,lda # saving memory

L_indices=list(range(lenX)) ##########


def transform_vocab(idx_doc_bow):
    global n_topics,ldamm
    bow_converted=ldamm[idx_doc_bow]
    local_array=[0]*(n_topics+1)
    for feat_tuple in bow_converted:
        local_array[feat_tuple[0]+1]=feat_tuple[1]
    return local_array
        
# Compute the feature vector :
p=Pool(10)
a=p.map(transform_vocab,L_indices)

cPickle.dump(a,open('sauvegarde_qui_sauve_la_vie.p','wb'))
topic_features_df=pd.DataFrame(a)
del a
topic_features_df=topic_features_df[topic_features_df.columns[1:]]
topic_features_df.columns=['lda_feature_'+str(i) for i in range(len(topic_features_df.columns))]
topic_features_df.to_csv('lda_features_'+str(n_topics)+'_'+str(name_file)+'_topics_df.csv',index=None)
#compute_lda_feature(n_topics,name_file)
t3=time.time()
print(t2-t1)
print(t3-t2)

285.679936171
582.179489851
